In [3]:
!pip install imutils

  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25862 sha256=4df05009385f6b58dc9eec4b4514713f4109ca05f581439f57b7d4032f2c29d0
  Stored in directory: c:\users\yogeshwaran\appdata\local\pip\cache\wheels\86\d7\0a\4923351ed1cec5d5e24c1eaf8905567b02a0343b24aa873df2
Successfully built imutils


In [7]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pickle-mixin

  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-py3-none-any.whl size=6002 sha256=96ef57064bc440dee6d48fec8b329f13b1bb79fb1c516cbd5cfa44b44c10c0ca
  Stored in directory: c:\users\yogeshwaran\appdata\local\pip\cache\wheels\d0\70\0b\673e09a7ed429660d22352a1b117b4f616a8fc054bdd7eb157
Successfully built pickle-mixin
Note: you may need to restart the kernel to use updated packages.


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from skimage import feature
from imutils import build_montages
from imutils import paths
import numpy as np
import cv2
import os
import pickle

In [8]:
trainingPath=r"D:\STUDIES\Project\Nalaya Thiran Dataset\dataset-20220920T082711Z-001\dataset\spiral\training"
testingPath=r"D:\STUDIES\Project\Nalaya Thiran Dataset\dataset-20220920T082711Z-001\dataset\spiral\testing"

In [17]:
def load_split ( path ) :
    #grab the list of images in the input directory , then initialize
    # the list of data ( i.e. , images ) and class labels
    imagePaths = list ( paths.list_images ( path ) )
    data = [ ]
    labels = [ ]
   #loop over the image paths
    for imagePath in imagePaths :
        #extract the class label from the filename
        label = imagePath.split ( os.path.sep ) [ -2 ]
        #load the input image , convert it to grayscale , and resize
        # it to 200x200 pixels , ignoring aspect ratio
        image = cv2.imread ( imagePath )
        image = cv2.cvtColor ( image , cv2.COLOR_BGR2GRAY )
        image = cv2.resize ( image , ( 200 , 200 ) )
        # threshold the image such that the drawing appears as white
        # on a black background
        image = cv2.threshold(image , 0 , 255 , cv2.THRESH_BINARY_INV | cv2 . THRESH_OTSU)[ 1 ]
        # quantify the image
        features = quantify_image(image)
        # update the data and labels lists , respectively
        data.append ( features )
        labels.append ( label )
    # return the data and labels
    return ( np.array ( data ) , np.array ( labels ) )

In [19]:
def quantify_image(image):
    # compute the histogram 
    features = feature.hog(image, orientations=9,
    pixels_per_cell=(10, 10), cells_per_block=(2, 2),
    transform_sqrt=True, block_norm="L1")

    # return the feature vector
    return features

In [20]:
(X_train, y_train)=load_split(trainingPath)
(X_test,y_test)=load_split(testingPath)

In [21]:
# encode the labels as integers
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
print(X_train.shape,y_train.shape)

(72, 12996) (72,)


In [22]:
print( " [ INFO ] training model " )
model = RandomForestClassifier(n_estimators = 100 )
model.fit(X_train , y_train)

 [ INFO ] training model 


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [89]:
testPaths = list(paths.list_images(testingPath))
idxs = np.arange(0 , len(testPaths))
print(idxs)
idxs = np.random.choice(idxs , size=(25,), replace=False)
print(idxs)
images = []

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
[20 28 17 25 13  5  3 14 29  0 15 18  9  1 24 21 22 12  6 27 23  4 11 10
 16]


In [90]:
for i in idxs:
    image = cv2.imread(testPaths[i])
    output = image.copy()
    output = cv2.resize(output , (128,128))
    image = cv2.cvtColor(image , cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image , (200,200))
    image = cv2.threshold(image , 0 , 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    features = quantify_image(image)
    preds = model.predict([features])
    label = le.inverse_transform(preds)[0]
    color = (0 , 255 , 0) if label == "healthy" else (0 , 0 ,255)
    cv2.putText(output , label , (3, 20) , cv2.FONT_HERSHEY_SIMPLEX , 0.5,color , 2)
    images.append(output)

In [92]:
montage = build_montages(images , (128,128) ,(5,5))[0]
cv2.imshow("OUTPUT",montage)
cv2.waitKey(0)

-1

In [93]:
predictions = model.predict(X_test)
cm = confusion_matrix(y_test , predictions).flatten()
print(cm)
(tn , fp , fn , tp) = cm
accuracy  = (tp + tn) / float(cm.sum())
print(accuracy)

[13  2  3 12]
0.8333333333333334


In [94]:
pickle.dump(model , open('parkinsons.pkl','wb'))